In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive
from hloc import reconstruction_belv
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

import h5py
import pickle
import pycolmap

In [2]:
dataset = Path('./epoch0')
images = Path(dataset / 'images/')
# outputs = Path(dataset)
# !rm -rf $outputs
sfm_pairs = dataset / 'pairs-sfm.txt'
loc_pairs = dataset / 'pairs-loc.txt'
sfm_dir = dataset
features = dataset / 'features.h5'
matches = dataset / 'matches.h5'

feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

In [ ]:
ref_images = [p.relative_to(images).as_posix() for p in (images).iterdir()]
print(len(ref_images), "images")
# plot_images([read_image(images / r) for r in references[:4]], dpi=50)

In [ ]:
# extract_features.main(feature_conf, images, image_list=references, feature_path=features)
# pairs_from_exhaustive.main(sfm_pairs, image_list=references)
# match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

In [ ]:
def build_h5datasets(matches_path, pair):
    from hloc.utils.io import (list_h5_names, get_matches, get_keypoints, find_pair) 
    from hloc.utils.parsers import names_to_pair
    import h5py
    
    # Retrieve matched features with SG and build HDF5 datasets
    with open(matches_path, 'rb') as f:
        mfeats = pickle.load(f)
    # print(mfeats)

    # Build hdf5 features datasets
    pair = names_to_pair(pair[0], pair[1])
    data = {'keypoints': mfeats[0]['mkpts0'],
           'descriptors': mfeats[0]['descr0'], 
           'scores': mfeats[0]['scores0']} 
    name = pair[0]
    with h5py.File(features, 'a') as fd:
        if name in fd:
            del fd[name]
        grp = fd.create_group(name)
        for k, v in data.items():
            grp.create_dataset(k, data=v)

    data = {'keypoints': mfeats[0]['mkpts1'],
           'descriptors': mfeats[0]['descr1'], 
           'scores': mfeats[0]['scores1']} 
    name = pair[1]
    with h5py.File(features, 'a') as fd:
        if name in fd:
            del fd[name]
        grp = fd.create_group(name)
        for k, v in data.items():
            grp.create_dataset(k, data=v)

    # Build hdf5 matching datasets
    matchArray = []
    matchScores = []
    npts = len(mfeats[0]['mkpts1'])
    for i in range(0, npts):
        matchArray.append(i)
        matchScores.append(1)

    pair = names_to_pair(pair[0], pair[1])
    with h5py.File(dataset / "matches.h5",'a') as fd:
        if pair in fd:
            del fd[pair]
        grp = fd.create_group(pair)
        grp.create_dataset('matches0', data=matchArray)
        grp.create_dataset('matching_scores0', data=matchScores)


sgmatches_path = Path(dataset / 'IMG_0520_IMG_2131_features.pickle')
build_h5datasets(sgmatches_path, ref_images)

In [ ]:
# Run reconstruction
pycolmap.CameraMode.__members__.keys()
pairs_from_exhaustive.main(sfm_pairs, image_list=ref_images)
model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, image_list=ref_images, verbose = True,  camera_mode=pycolmap.CameraMode.PER_IMAGE)
model.write_text(str(sfm_dir))

In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping")
fig.show()
visualization.visualize_sfm_2d(model, images, color_by='depth', n=2)

In [ ]:
for image_id, image in model.images.items():
    print(image_id, image)
# for point3D_id, point3D in reconstructionObj.points3D.items():
#     print(point3D_id, point3D)
for camera_id, camera in model.cameras.items():
    print(camera_id, camera)

In [ ]:
#
# import os
# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument("--database_path", default="database.db")
# args = parser.parse_args()

# if os.path.exists(args.database_path):
#     print("ERROR: database path already exists -- will not modify it.")
#     return

In [ ]:
## Run recosnstruction with custom camera
pairs_from_exhaustive.main(sfm_pairs, image_list=ref_images)

# Define cameras
model = "FULL_OPENCV"
width = 6000
height = 4000
params1 = np.array((10145.828770984808, 10181.29619861736,  2928.763582703865, 1636.663975450278, -0.04643331697636249, 0.7615948374750167, -0.010397811196689507, -0.002202286738591026, -4.287589186049892, 0., 0., 0.))
params2 = np.array((6900.766178626993, 6919.0517432373235, 3055.9219427396583, 1659.8768050681379, -0.07241143420209739, 0.00311945599198001, -0.008597066196675609, 0.002601995972163532, 0.46863386164346776, 0., 0., 0.))
camera1_dict = {
    'model': model,
    'width': width,
    'height': height,
    'params': params1
}
camera2_dict = {
    'model': model,
    'width': width,
    'height': height,
    'params': params2
}

# model = reconstruction_belv.main(sfm_dir, images, sfm_pairs, features, matches, image_list=references, verbose = True,  camera_mode=pycolmap.CameraMode.PER_IMAGE)

In [ ]:
from hloc.reconstruction_belv import create_empty_db

database_path = sfm_dir / 'database.db'
create_empty_db(database_path)

In [ ]:
# void import_images(const py::object database_path_,
#                    const py::object image_path_,
#                    const CameraMode camera_mode,
#                    const std::string camera_model,
#                    const std::vector<std::string> image_list)

# cam1 = pycolmap.Camera(
#     model= model,
#     width= width,
#     height= height,
#     params= params1,
# )
 
from hloc.reconstruction_belv import get_image_ids
from hloc.triangulation_belv import (import_features,import_matches)

def import_images(image_dir, database_path, camera_mode, camera_model, image_list=None):
    logger.info('Importing images into the database...')
    images = list(image_dir.iterdir())
    if len(images) == 0:
        raise IOError(f'No images found in {image_dir}.')
    with pycolmap.ostream():
        pycolmap.import_images(database_path, image_dir, camera_mode, camera_model,
                               image_list=image_list or [])    

import_images(sfm_dir, database_path, 'PER_IMAGE', 'FULL_OPENCV', ref_images)
image_ids = get_image_ids(database_path)    
import_features(image_ids, database_path, features)
# import_matches(image_ids, database_path, sfm_pairs, matches)       

In [ ]:
import os
import database
from hloc import logger
from hloc.utils.database import COLMAPDatabase
from hloc.reconstruction_belv import create_empty_db

assert features.exists(), features
assert sfm_pairs.exists(), sfm_pairs
assert matches.exists(), matches

database_path = sfm_dir / 'database.db'
create_empty_db(database_path)

# sfm_dir.mkdir(parents=True, exist_ok=True)
# database_path = sfm_dir / 'database.db'
# # Open the database (remove previous one, if already exists)
# if os.path.exists(database_path):
#     os.remove(database_path)
    
# if database_path.exists():
#     logger.warning('The database already exists, deleting it.')
#     database_path.unlink()
# logger.info('Creating new database...')

database = COLMAPDatabase.connect(database_path)
database.create_tables()

# Create Cameras
camera_id1 = database.add_camera(camera1_dict['model'], camera1_dict['width'], 
                           camera1_dict['height'], camera1_dict['params'])
camera_id2 = database.add_camera(camera2_dict['model'], camera2_dict['width'], 
                           camera2_dict['height'], camera2_dict['params'])

# Create images.
image_id1 = database.add_image(str(images/ref_images[0]), camera_id1)
image_id2 = database.add_image(str(images/ref_images[1]), camera_id2)

# Commit the data to the file.
database.commit()
database.close()

from hloc.reconstruction_belv import get_image_ids
from hloc.triangulation_belv import (import_features,import_matches)

image_ids = get_image_ids(database_path)    
import_features(image_ids, database_path, features)
import_matches(image_ids, database_path, sfm_pairs, matches)
# if not skip_geometric_verification:
#     geometric_verification(database, pairs, verbose)


# from hloc.utils.io import (get_keypoints, get_matches)
# def import_matches(image_ids, database_path, pairs_path, matches_path,
#                    min_match_score=None, skip_geometric_verification=False):
#     logger.info('Importing matches into the database...')
    
#     with open(str(pairs_path), 'r') as f:
#         pairs = [p.split() for p in f.readlines()]
    
#     db = COLMAPDatabase.connect(database_path)
#     matched = set()
    
#     for name0, name1 in pairs:
#         images_path = [path for path, v in image_ids.items()]
#         id0, id1 = image_ids[images_path[0]], image_ids[images_path[1]]
#         if len({(id0, id1), (id1, id0)} & matched) > 0:
#             continue
#         matches, scores = get_matches(matches_path, name0, name1)
#         if min_match_score:
#             matches = matches[scores > min_match_score]
#         db.add_matches(id0, id1, matches)
#         matched |= {(id0, id1), (id1, id0)}

#         if skip_geometric_verification:
#             db.add_two_view_geometry(id0, id1, matches)

#     db.commit()
#     db.close()

In [ ]:
from hloc.reconstruction_belv import run_reconstruction

reconstruction = run_reconstruction(sfm_dir, database_path, sfm_pairs, verbose=True)

In [ ]:
import database

database_path =  str(dataset / "database.db")

# Open the database (remove previous one, if already exists)
if os.path.exists(database_path):
    os.remove(database_path)
    
db = database.COLMAPDatabase.connect(database_path)

# Try creating all the tables upfront.
db.create_tables()

# Create cameras.
model = "FULL_OPENCV"
width = 6000
height = 4000
params1 = np.array((10145.828770984808, 10181.29619861736,  2928.763582703865, 1636.663975450278, -0.04643331697636249, 0.7615948374750167, -0.010397811196689507, -0.002202286738591026, -4.287589186049892, 0., 0., 0.))
params2 = np.array((6900.766178626993, 6919.0517432373235, 3055.9219427396583, 1659.8768050681379, -0.07241143420209739, 0.00311945599198001, -0.008597066196675609, 0.002601995972163532, 0.46863386164346776, 0., 0., 0.))

camera1_dict = {
    'model': model,
    'width': width,
    'height': height,
    'params': params1
}
camera21_dict = {
    'model': model,
    'width': width,
    'height': height,
    'params': params2
}

camera_id1 = db.add_camera(model, width, height, params1)
camera_id2 = db.add_camera(model, width, height, params2)

# Create images.
image_id1 = db.add_image("image1.png", camera_id1)
image_id2 = db.add_image("image2.png", camera_id2)

# Create dummy keypoints.
# Note that COLMAP supports:
#      - 2D keypoints: (x, y)
#      - 4D keypoints: (x, y, theta, scale)
#      - 6D affine keypoints: (x, y, a_11, a_12, a_21, a_22)

num_keypoints = 1000
keypoints1 = np.random.rand(num_keypoints, 2) * (width1, height1)
keypoints2 = np.random.rand(num_keypoints, 2) * (width1, height1)
db.add_keypoints(image_id1, keypoints1)
db.add_keypoints(image_id2, keypoints2)

# Create dummy matches.
M = 50
matches12 = np.random.randint(num_keypoints, size=(M, 2))
matches23 = np.random.randint(num_keypoints, size=(M, 2))
matches34 = np.random.randint(num_keypoints, size=(M, 2))
db.add_matches(image_id1, image_id2, matches12)

# Commit the data to the file.
db.commit()

In [ ]:
from database import (blob_to_array, image_ids_to_pair_id)

# Read and check cameras.
rows = db.execute("SELECT * FROM cameras")
camera_id, model, width, height, params, prior = next(rows)
params = blob_to_array(params, np.float64)
assert camera_id == camera_id1
assert model == model1 and width == width1 and height == height1
assert np.allclose(params, params1)

camera_id, model, width, height, params, prior = next(rows)
params = blob_to_array(params, np.float64)
assert camera_id == camera_id2
assert model == model2 and width == width2 and height == height2
assert np.allclose(params, params2)

# Read and check keypoints.
keypoints = dict(
    (image_id, blob_to_array(data, np.float32, (-1, 2)))
    for image_id, data in db.execute(
        "SELECT image_id, data FROM keypoints"))
assert np.allclose(keypoints[image_id1], keypoints1)
assert np.allclose(keypoints[image_id2], keypoints2)

In [ ]:
for pair_id, data in db.execute("SELECT pair_id, data FROM matches"):
    print(pair_id)

In [ ]:
from database import (blob_to_array, image_ids_to_pair_id, pair_id_to_image_ids)


# Read and check matches.
pair_ids = [image_ids_to_pair_id(image_id1, image_id2)]

matches = dict(
    (pair_id_to_image_ids(pair_id),
     blob_to_array(data, np.uint32, (-1, 2)))
    for pair_id, data in db.execute("SELECT pair_id, data FROM matches")
)

assert np.all(matches[(image_id1, image_id2)] == matches12)

# Clean up.
db.close()

